In [1]:
from spimy2 import BSBIndexConstuction
import pandas as pd

[nltk_data] Downloading package punkt to /home/ambar/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [5]:
from spimi import SPIMI

[nltk_data] Downloading package punkt to /home/ambar/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
# testeo
path = './dataset/'
data_path = path + 'spotify_songs_test.csv'

data = pd.read_csv(data_path)

In [3]:

merged_files = BSBIndexConstuction(data)

Block 0 written to ./blocks/block_0.txt
Block 1 written to ./blocks/block_1.txt
Block 2 written to ./blocks/block_2.txt
Block 3 written to ./blocks/block_3.txt
Block 4 written to ./blocks/block_4.txt
Block 5 written to ./blocks/block_5.txt


In [6]:
spimi = SPIMI(size_per_block=10240*4,
              output_folder='./blocks/',
              ram_limit=1024*1024*1024,
              size_per_block_out= 1024*5.9)

spimi.BSBIndexConstuction(data)

['./blocks/block_0.txt',
 './blocks/block_1.txt',
 './blocks/block_2.txt',
 './blocks/block_3.txt',
 './blocks/block_4.txt',
 './blocks/block_5.txt']